# Atividade aula 3
Grupo composto: 

Fabiano Thiele Fracassi
Ricardo Schmitt Martins,
Roberto Emilio de Senna,
Roberto Rodrigues Barbosa

In [ ]:
import pandas as pd
import seaborn as sns
from bcb import sgs, currency, TaxaJuros

## Informe de Resultados do MGLU3

In [ ]:
## Importa os dados do google drive
bp = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTq_-oCydZOtrO1J_eMkG849ttZbQ_N4kLSOSFWKIuTwQZ9FMEHjN1mlzK4c_Ba9xLUNf6s67PhXf9F/pub?gid=86692319&single=true&output=csv',
               header=1)
dre = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTq_-oCydZOtrO1J_eMkG849ttZbQ_N4kLSOSFWKIuTwQZ9FMEHjN1mlzK4c_Ba9xLUNf6s67PhXf9F/pub?gid=2092096157&single=true&output=csv',
                    header=1)

In [ ]:
## Transforma os dados de forma a deixar um dataframe limpo para os cálculos
# Trabalha com o dataframe do Balanço Patrimonial
bpr = bp.transpose().iloc[:,[4,5,27]]
bpr.rename(columns={4:'clientes',5:'estoques',27:'fornecedores'},inplace=True)
bpr.drop(index='Unnamed: 0',inplace=True)
#display(bpr.head())

# Trabalha com o dataframe da DRE
drer = dre.transpose().iloc[:,[2,3]]
drer.rename(columns={2:'receita_liq',3:'custos'},inplace=True)
drer.drop(index=' ',inplace=True)
#display(drer.head())

# Combina os dois dataframes
df = pd.concat([bpr.transpose(),drer.transpose()]).transpose()
df = df.astype({'clientes':float,'estoques':float,'fornecedores':float,'receita_liq':float,'custos':float}) 
#display(df.head())

# Converte o índice das colunas em formato data
df = df.reset_index()
df = df.rename(columns={'index':'data'})
df['data']=pd.to_datetime(df['data'].astype(str), format='%d/%m/%Y')

In [ ]:
## Efetua os cálculos dos indicadores
df['custos']=abs(df['custos'])
df['PME'] = (df['estoques'] / df['custos']) * 90
df['PMR'] = (df['clientes'] / df['receita_liq']) * 90
df['PMP'] = (df['fornecedores'] / df['custos']) * 90
df['CO'] = df['PME'] + df['PMR']
df['CC'] = df['CO'] - df['PMP']
df.head()

,data,clientes,estoques,fornecedores,receita_liq,custos,PME,PMR,PMP,CO,CC
0,2022-06-30,5621703.0,7965704.0,7380799.0,8562389.0,6109614.0,117.341842,59.090199,108.725676,176.432041,67.706365
1,2022-03-31,4801071.0,8077255.0,6248478.0,8762175.0,6330426.0,114.834760,49.313828,88.834941,164.148588,75.313647
2,2021-12-31,5650759.0,9112214.0,10098944.0,9399998.0,7025695.0,116.728560,54.103023,129.368690,170.831583,41.462893
3,2021-09-30,5427841.0,8126343.0,9177531.0,8612033.0,6877900.0,106.336363,56.723620,120.091567,163.059983,42.968416
4,2021-06-30,3779102.0,7496866.0,8241821.0,9013308.0,6705039.0,100.628489,37.735222,110.627826,138.363711,27.735884


In [ ]:
## Cria um novo dataframe apenas com as colunas dos indicadores
df2 = df.loc[:,['PME','PMR','PMP','CO','CC']]
df2.head().round(1)

,PME,PMR,PMP,CO,CC
0,117.3,59.1,108.7,176.4,67.7
1,114.8,49.3,88.8,164.1,75.3
2,116.7,54.1,129.4,170.8,41.5
3,106.3,56.7,120.1,163.1,43.0
4,100.6,37.7,110.6,138.4,27.7


## Indicadores Financeiros do Banco Central

In [ ]:
# Busca a série da SELIC no SGS
selic = sgs.get({'selic':432}, start = '2012-01-01')
# Busca a série do IPCA e IGP-M
inflacao = sgs.get({'ipca':433, 'igp-m':189}, start = '2012-01-01')
# Transforma a frequência da data em mensal
inflacao.index = inflacao.index.to_period('M')
# Importas moedas 
moeda = currency.get(['USD', 'EUR'], start='2012-01-01', end='2022-11-16')
# Importa Taxa média de juros das operações de crédito - Pessoas jurídicas - Total - %a.a.
juros = sgs.get({'juros_aa':20715}, start = '2012-01-01')

In [ ]:
def adjust_format(df):
    df = df.sort_values('Date',ascending=False).reset_index()
    df['Date'] = pd.to_datetime(df['Date'].astype(str), format='%Y-%m-%d')
    df['datenum'] = df['Date'].values.astype(float) / 1e10
    df.set_index(['datenum'], inplace=True)
    return df

In [ ]:
s=adjust_format(selic)
j=adjust_format(juros).drop('Date',axis=1)
m=adjust_format(moeda).drop('Date',axis=1)

In [ ]:
df3 = pd.concat([s, j, m], axis=1, join='outer')
df3['EUR']=df3['EUR'].fillna(method='ffill')
df3['USD']=df3['USD'].fillna(method='ffill')
df3['juros_aa']=df3['juros_aa'].fillna(method='bfill')
df3['juros_aa']=df3['juros_aa'].fillna(method='ffill')
df3

,Date,selic,juros_aa,EUR,USD
datenum,,,,,
166864320.0,2022-11-17,13.75,19.01,NaN,NaN
166855680.0,2022-11-16,13.75,19.01,5.5364,5.3204
166847040.0,2022-11-15,13.75,19.01,5.5364,5.3204
166838400.0,2022-11-14,13.75,19.01,5.4773,5.3013
166829760.0,2022-11-13,13.75,19.01,5.4773,5.3013
...,...,...,...,...,...
132572160.0,2012-01-05,11.00,19.06,2.3511,1.8371
132563520.0,2012-01-04,11.00,19.06,2.3677,1.8272
132554880.0,2012-01-03,11.00,19.06,2.4057,1.8457


In [ ]:
df['datenum'] = df['data'].values.astype(float) / 1e10
df.set_index(['datenum'], inplace=True)
dados=pd.concat([df,df3], axis=1, join='inner')
dados = dados.drop('Date',axis=1)
dados.reset_index(drop=True,inplace=True)

In [ ]:
dados

,data,clientes,estoques,fornecedores,receita_liq,custos,PME,PMR,PMP,CO,CC,selic,juros_aa,EUR,USD
0,2022-06-30,5621703.0,7965704.0,7380799.0,8562389.0,6109614.0,117.341842,59.090199,108.725676,176.432041,67.706365,13.25,19.51,5.4842,5.2380
1,2022-03-31,4801071.0,8077255.0,6248478.0,8762175.0,6330426.0,114.834760,49.313828,88.834941,164.148588,75.313647,11.75,18.66,5.2561,4.7378
2,2021-12-31,5650759.0,9112214.0,10098944.0,9399998.0,7025695.0,116.728560,54.103023,129.368690,170.831583,41.462893,9.25,17.18,6.3210,5.5805
3,2021-09-30,5427841.0,8126343.0,9177531.0,8612033.0,6877900.0,106.336363,56.723620,120.091567,163.059983,42.968416,6.25,14.84,6.2983,5.4394
4,2021-06-30,3779102.0,7496866.0,8241821.0,9013308.0,6705039.0,100.628489,37.735222,110.627826,138.363711,27.735884,4.25,12.72,5.9276,5.0022
5,2021-03-31,3714318.0,6808443.0,7070516.0,8252813.0,6182711.0,99.108606,40.506021,102.923530,139.614628,36.691098,2.75,12.11,6.6915,5.6973
6,2020-12-31,4761899.0,5927236.0,8501398.0,10065793.0,7580243.0,70.373897,42.576964,100.936846,112.950861,12.014015,2.00,11.09,6.3779,5.1967
7,2020-09-30,4033864.0,5005934.0,6104258.0,8308326.0,6129592.0,73.501476,43.696860,89.628024,117.198336,27.570312,2.00,9.94,6.6132,5.6407
8,2020-06-30,4386122.0,4198175.0,5333983.0,5568245.0,4133041.0,91.418341,70.893249,116.151393,162.311590,46.160197,2.25,10.81,6.1539,5.4760
9,2020-03-31,2147035.0,4075499.0,4132662.0,5234749.0,3814276.0,96.163704,36.913546,97.512498,133.077251,35.564753,3.75,13.59,5.7264,5.1987
